# 🧠 Week 07-08 · Notebook 01 · LangChain Essentials for Manufacturing Copilots

Lay the foundation for building LangChain pipelines that power maintenance, quality, and EHS assistants across Arvind Manufacturing's plants.

## 🎯 Learning Objectives
- Understand LangChain core abstractions (prompt, LLM, chains, tools).
- Connect LangChain to the synthetic manufacturing corpus prepared in Weeks 05-06.
- Prototype a maintenance triage chain with lineage logging for audits.
- Document lifecycle checkpoints that align with corporate governance.

## 🧩 Scenario
The Pune plant wants a first-class troubleshooting assistant exposing SOP summaries and IoT alerts. LangChain needs to be evaluated for rapid prototyping while meeting ISO 9001 change control.

## 🧱 LangChain Building Blocks
| Component | Manufacturing Use Case | Notes |
| --- | --- | --- |
| `PromptTemplate` | Standardize safety disclaimers | Tag with revision IDs |
| `ChatOpenAI` | Generate bilingual responses | Enforce completion tokens ceiling |
| `LLMChain` | Maintenance triage | Log run metadata to MLflow |
| `Tool` | Spare part lookup, downtime analytics | Wrap plant REST APIs |
| `Memory` | Persist shift conversation | Store in Redis with TTL = 24h |

In [ ]:
# Minimal LangChain pipeline using synthetic maintenance data
from pathlib import Path
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Ensure the SOP directory and file exist
SOP_DIR = Path('data/sop_manuals')
SOP_DIR.mkdir(parents=True, exist_ok=True)
SOP_PATH = SOP_DIR / 'punch_press_v7.txt'
if not SOP_PATH.exists():
    SOP_PATH.write_text("""
Standard Operating Procedure: Punch Press v7

Safety First: Always wear safety glasses and gloves. Ensure machine is locked out before maintenance.

Lubrication Schedule:
- Daily: Check hydraulic fluid levels.
- Weekly: Lubricate punch press bearings with Grade 2 lithium grease.
- Monthly: Inspect and lubricate all moving parts.
""", encoding='utf-8')

sample_context = SOP_PATH.read_text(encoding='utf-8')[:800]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful Manufacturing Copilot that references SOPs for maintenance guidance."),
    ("human", "Context: {context}

Question: {question}")
])

# Using LangChain Expression Language (LCEL)
chain = prompt | ChatOpenAI(model='gpt-4o-mini', temperature=0.2) | StrOutputParser()

response = chain.invoke({
    "context": sample_context,
    "question": "How often should I lubricate the punch press bearings?"
})

print(response)

### 🔒 Governance Inline
- Record prompt, input, and output hashes for audit trail.
- Tag runs with plant, shift, and technician supervisor ID.
- Enforce translation to Spanish when `technician_language = es`.

In [ ]:
from hashlib import sha256
import json
from datetime import datetime
import time

# --- Governance and Logging Wrapper ---
def execute_and_log_chain(chain, inputs):
    start_time = time.time()
    
    # Execute the chain
    response = chain.invoke(inputs)
    
    latency_ms = (time.time() - start_time) * 1000
    
    # Create governance record
    run_evidence = {
        "timestamp": datetime.now().isoformat(),
        "plant_id": "PUNE-01",
        "shift": "morning",
        "supervisor_id": "SUP-4872",
        "prompt_hash": sha256(str(chain.get_prompts()).encode()).hexdigest()[:12],
        "input_hash": sha256(inputs['context'].encode()).hexdigest()[:12],
        "output_hash": sha256(response.encode()).hexdigest()[:12],
        "model": chain.middle[0].model_name,
        "latency_ms": round(latency_ms, 2)
    }
    
    # Append log to a file
    log_dir = Path('governance')
    log_dir.mkdir(exist_ok=True)
    with open(log_dir / 'langchain_run_log.jsonl', 'a') as f:
        f.write(json.dumps(run_evidence) + '\n')
        
    return response, run_evidence

# --- Execute with Logging ---
inputs = {
    "context": sample_context,
    "question": "How often should I lubricate the punch press bearings?"
}
final_response, evidence = execute_and_log_chain(chain, inputs)

print("--- Chain Response ---")
print(final_response)
print("\n--- Governance Log ---")
print(json.dumps(evidence, indent=2))

## 🧪 Lab Assignment
1. **Access Real SOP Data**: Create a function that loads `data/sop_manuals/punch_press_v7.txt` or creates the directory and file with sample content if it doesn't exist. Use Path from pathlib for platform-independent paths.

2. **Implement Bilingual Support**: Extend the chain to detect language preference and translate responses when `technician_language = "es"`. Use this prompt template:
   ```python
   bilingual_prompt = ChatPromptTemplate.from_messages([
       ("system", "You are a bilingual Manufacturing Copilot. If technician_language is 'es', respond in Spanish."),
       ("human", "Context: {context}\nQuestion: {question}\nTechnician language: {technician_language}")
   ])
   ```

3. **Add Comprehensive Logging**: Extend the `run_evidence` dictionary to include these exact fields:
   - `completion_tokens`: Count of tokens in the response
   - `latency_ms`: Time taken for chain execution
   - `prompt_version`: SOP manual version (extract from filename)
   - `safety_disclaimer_present`: Boolean check if response contains safety warning
   
4. **Create MLflow Integration**: Write a function that logs the metrics to MLflow using this pattern:
   ```python
   import mlflow
   
   def log_chain_metrics(run_evidence):
       mlflow.start_run()
       for key, value in run_evidence.items():
           if isinstance(value, (int, float)):
               mlflow.log_metric(key, value)
           else:
               mlflow.log_param(key, value)
       mlflow.end_run()
   ```

## ✅ Checklist
- [ ] SOP data loading function implemented with proper error handling
- [ ] Bilingual chain responds correctly in Spanish when requested
- [ ] Governance log contains all required fields (completion_tokens, latency_ms, etc.)
- [ ] MLflow integration successfully records metrics
- [ ] Chain outputs contain appropriate safety disclaimers for maintenance tasks

## 📚 References
- LangChain Docs — [LLMChain](https://python.langchain.com/docs/modules/model_io/chains/)
- ISO 9001 Change Control Guidelines
- Week 05-06 Data Profiling Notebook